### Regression Tree

In [1]:
import numpy as np
import sys

### Get data from txt files

In [2]:
def get_data():
    train_data = np.genfromtxt('./data/housing_train.txt')
    test_data = np.genfromtxt('./data/housing_test.txt')

    return train_data, test_data

In [3]:
# get data from the txt files
train, test = get_data()

### Normalization using shift/scale

Normalization is done on both test and train data.

In [4]:
def normalize(data_set, train_len):
    # normalize data using shift/scale
    maxs = np.max(data_set, axis=0)
    mins = np.min(data_set, axis=0)

    maxs = maxs[:-1]
    mins = mins[:-1]

    for feature in range(len(data_set[0])-1):
        for entry in data_set:
            entry[feature] = (entry[feature] - mins[feature]) / (maxs[feature] - mins[feature])

    return data_set[:train_len], data_set[train_len:]

In [5]:
# combine test and train data to perform normalization
full_data = np.concatenate((train, test), axis=0)

# normalize the data
train, test = normalize(full_data, len(train))

In [6]:
def get_mean_labels(dataset):
    return np.mean(dataset, axis=0)

### Get all possible Thresholds

Returns thresholds for all features by calculating midpoints for each feature value

In [7]:
def get_thresholds(dataset):

    ts = []
    for feature in range(len(dataset[0])-1):
        t = []
        for entry in range(len(dataset) - 1):
            t.append((dataset[entry][feature] + dataset[entry+1][feature]) / 2)
        ts.append(t)

    return np.array(ts)

### Get best **Feature** and **Threshold** pair for Splitting

In [8]:
def get_best_split(dataset, thresholds):
    
    best_feature = 0
    best_threshold = 0
    max_info_gain = 0
    
    mse_before = get_mse(dataset)
    
    for feature in range(len(dataset[0])-1):
        for threshold in thresholds[feature]:
            left, right = split_data(dataset, feature, threshold)
            
            if len(left) == 0 or len(right) == 0:
                continue
                
            left_mse = get_mse(left)
            right_mse = get_mse(right)
            
            w = len(left) / len(dataset)
            mse_after = (w * left_mse) + ((1-w) * right_mse)
            
            info_gain = mse_before - (left_mse + right_mse)

            if max_info_gain <= info_gain:
                max_info_gain = mse_after
                best_feature = feature
                best_threshold = threshold

    return best_feature, best_threshold, max_info_gain

### Terminal Nodes

Represents the leaf nodes of the Regression Tree that returns the actual predictions by calculating mean label over all datapoints in the node.

In [9]:
class Terminal:

    def __init__(self, dataset):
        predicts = get_mean_labels(dataset)
        self.prediction = predicts[-1]

    def predict(self):
        return self.prediction

### Nodes

Represents a single Node in the Regression Tree which contains the splitting criteria denoted by a (Feature, Threshold) pair.

Also, contains a link to the left and right node.

In [10]:
class Node:

    def __init__(self, feature, threshold, left_node, right_node):
        self.feature = feature
        self.threshold = threshold
        self.left_node = left_node
        self.right_node = right_node

### Mean Squared Error

In [11]:
def get_mse(dataset):

    prediction = get_mean_labels(dataset)
    mse = 0

    for entry in dataset:
        mse += np.square(entry[-1] - prediction[-1])

    return mse

### Split the data in two bins

Splits the data based on the given feature and threshold pair

In [12]:
def split_data(dataset, feature, threshold):
    left = [entry for entry in dataset if entry[feature] < threshold]
    right = [entry for entry in dataset if entry[feature] >= threshold]
    return left, right

### Building Tree Recursively

Builds the Regression Tree and returns the root of the Tree as a model.

In [13]:
def build_tree(dataset, depth):

    thresholds = get_thresholds(dataset)    
    best_feature, best_threshold, info_gain = get_best_split(dataset, thresholds)
    
    if info_gain == 0 or depth > 20:
        return Terminal(dataset)
    
    left_data, right_data = split_data(dataset, best_feature, best_threshold)

    left_node = build_tree(left_data, depth+1)
    right_node = build_tree(right_data, depth+1)

    return Node(best_feature, best_threshold, left_node, right_node)

In [14]:
## CREATE REGRESSION TREE

model = build_tree(train, 0)

### Print Decision Tree

In [15]:
def print_tree(root):

    if isinstance(root, Terminal):
        print('Prediction: {}'.format(root.predict()))
        return

    print('({}, {})'.format(root.feature, root.threshold))

    print('Left')
    print_tree(root.left_node)
    print('Right')
    print_tree(root.right_node)

In [ ]:
## PRINT OBTAINED MODEL

print_tree(model)

### Regress

Returns the label using learned model

In [17]:
def regress(root, entry):
    
    if isinstance(root, Terminal):
        return root.predict()
    
    if entry[root.feature] < root.threshold:
        result = regress(root.left_node, entry)
    else:
        result = regress(root.right_node, entry)
        
    return result

### Testing our Regression Tree

Returns mean squared error for the learned model applied over given test data

In [18]:
def test_model(model, test_data):
    
    predictions = []
    
    for entry in test_data:
        predictions.append(regress(model, entry))

    errors = []
    
    for i,p in enumerate(predictions):
        errors += np.square(test_data[i][-1] - p)
    
    mse = pd.Series(errors).mean()
    
    return mse

In [19]:
final_mse = test_model(model, test)
print(final_mse)

NameError: name 'pd' is not defined

22.867205542725184